<a href="https://colab.research.google.com/github/alessandronicolini/IncrementalLearning/blob/main/mnemonics_ALESSANDRO_prima_prova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np
import torch
from torchvision import transforms
from torchvision.datasets import VisionDataset
from PIL import Image
import random
import torchvision
ROOT = './data'
class ilCIFAR100(VisionDataset):
    """
    Extends CIFAR100 class. Split the dataset into 10 batches, each one containing 10 classes.
    You can retrieve the batches from the attribute "batches", it has different structure according to
    test and train CIFAR100 splits:
        - train -> batches is a dictionary {0:{'train':indexes, 'val':indexes}...} 
        - test -> batches is a dictionary {0:indexes...}
    where the keys are the batch number.

    Args:
        root (string): Root directory of dataset where directory
            `cifar-10-batches-py` exists or will be saved to if download is set to True.
        seed(int): used to ensure reproducibility in shuffling operations.
        val_size(float, optional): between 0 and 1, fraction of data used for validation.
        train (bool, optional): If True, creates dataset from training set, otherwise
            creates from test set.
        transform (callable, optional): A function/transform that takes in an PIL image
            and returns a transformed version. E.g, `transforms.RandomCrop`
        target_transform (callable, optional): A function/transform that takes in the
            target and transforms it.
        download (bool, optional): If true, downloads the dataset from the internet and
            puts it in root directory. If dataset is already downloaded, it is not
            downloaded again.
    """
    def __init__(self,classes_per_batch, seed, val_size=0.1, train=True, transform=None, target_transform=False, 
    download=True):
        
        super(ilCIFAR100, self).__init__(root=0)
        self.classes_per_batch=classes_per_batch

        
        self.__rs = seed # set random seed 
        self.train=train
        self.__transform_train = transforms.Compose([
            transforms.RandomCrop(32, padding=4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
        ])

        self.__transform_test = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
        ])
        # if train:
        #     self.batches = self.__make_train_batches(val_size)
        # else:
        #     self.batches = self.__make_test_batches()
        if self.train == 'train':
          self.dataset = torchvision.datasets.CIFAR100(root=ROOT, train=True,
                                            download=True, transform=self.__transform_train)
        elif self.train == 'exemplars':
          self.dataset = torchvision.datasets.CIFAR100(root=ROOT, train=True,
                                            download=True, transform=self.__transform_test)
        else:
          self.dataset = torchvision.datasets.CIFAR100(root=ROOT, train=False,
                                        download=True, transform=self.__transform_test)
          
        self.targets = np.array(self.dataset.targets) # make targets an array to exploit masking
        random.seed(seed)
        self.classes = random.sample(range(0, 100), 100)
        #self.classes = self.classes.reshape((10, -1)) # each row contains the classes for the corrisponding batch
        #print(self.classes)
        self.__dictionary = {}
        for i, c in enumerate(self.classes):
          self.__dictionary[c] = i


    def get_dict(self):
      return self.__dictionary
    def __getitem__(self, index):
        return index,self.dataset.__getitem__(index)[0],self.dataset.__getitem__(index)[1]
    def __len__(self):
        return self.dataset.__len__()
    def getbatches(self):
      classlist=self.classes
      batches=[]
      for i in range(0,int(100/self.classes_per_batch)):
        #print(i)
        batch=classlist[int(i*self.classes_per_batch):int(i*10+self.classes_per_batch)]
        batches.append(batch)
      return batches
    def get_batch_indexes(self):
      classlist=self.classes
      numclass=self.classes_per_batch
      batch_indexes=[]
      for i in range(0,int(100/self.classes_per_batch)):
        batch=classlist[int(i*numclass):int(i*numclass+numclass)]
        mask=np.isin(self.targets,batch)
        indexes=np.array(np.arange(len(self.dataset.targets)))
        indexes=indexes[mask]
        batch_indexes.append(indexes)
      return batch_indexes
    def get_class_indexes(self,label):
      indexes = np.array(np.arange(len(self.dataset.targets)))
      labels = self.dataset.targets
      mask = np.isin(labels, label)
      indexes = indexes[mask]

      return indexes
    def get_train_val(self,valid):
      batches=self.get_batch_indexes()
      train=[]
      val=[]
      for batch in batches:
        #print(type(batch))
        random.shuffle(batch)
        valbatch=batch[0:int(valid*len(batch))]
        trainbatch=batch[int(valid*len(batch)):]
        train.append(trainbatch)
        val.append(valbatch)
      return train,val
import torch
import torch.optim as optim
import torchvision
import time
import os
import argparse
import numpy as np

def tensor2im(input_image, imtype=np.uint8):
    mean = [0.5071, 0.4867, 0.4408]
    std = [0.2675, 0.2565, 0.2761]
    if not isinstance(input_image, np.ndarray):
        if isinstance(input_image, torch.Tensor):
            image_tensor = input_image.data
        else:
            return input_image
        image_numpy = image_tensor.cpu().detach().float().numpy()
        if image_numpy.shape[0] == 1:
            image_numpy = np.tile(image_numpy, (3, 1, 1))
        for i in range(len(mean)): 
            image_numpy[i] = image_numpy[i] * std[i] + mean[i]
        image_numpy = image_numpy * 255
        image_numpy = np.transpose(image_numpy, (1, 2, 0))
    else:
        image_numpy = input_image
    return image_numpy.astype(imtype)

In [7]:
!pip3 install 'import_ipynb'
!pip3 install 'tqdm'

!rm -r IncrementalLearning
# upload work files from your git hub repository
import sys

!git clone https://github.com/alessandronicolini/IncrementalLearning.git # clone proj repository
!rm -rf IncrementalLearning/README.md 
!rm -rf IncrementalLearning/baselines.ipynb

path = 'IncrementalLearning/'
if path not in sys.path:
    sys.path.append('IncrementalLearning/')

!pip3 install import_ipynb

Cloning into 'IncrementalLearning'...
remote: Enumerating objects: 60, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 534 (delta 36), reused 0 (delta 0), pack-reused 474
Receiving objects: 100% (534/534), 524.73 KiB | 7.72 MiB/s, done.
Resolving deltas: 100% (311/311), done.


In [5]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
from PIL import Image
import torchvision
import torchvision.transforms as transforms
import math
from sklearn.preprocessing import normalize
import copy
import torchvision.datasets as dsets
import torchvision.models as models
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from torch.utils.data import Subset, DataLoader, Dataset
import random
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import pandas as pd

import import_ipynb
#from IncrementalLearning.cifar100 import ilCIFAR100
from google.colab import output

from IncrementalLearning.resnet_cifar import resnet32
from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

importing Jupyter notebook from /content/IncrementalLearning/resnet_cifar.ipynb


In [62]:
def process_inputs_fp(tg_model, inputs, fusion_mode=False, feature_mode=False):
  tg_model_group1 = [tg_model.conv1, tg_model.bn1, tg_model.relu, tg_model.layer1]
  tg_model_group1 = nn.Sequential(*tg_model_group1)
  tg_fp1 = tg_model_group1(inputs)
  fp1 = tg_fp1
  tg_model_group2 = tg_model.layer2
  tg_fp2 = tg_model_group2(fp1)
  fp2 = tg_fp2
  tg_model_group3 = [tg_model.layer3, tg_model.avgpool]
  tg_model_group3 = nn.Sequential(*tg_model_group3)
  tg_fp3 = tg_model_group3(fp2)
  fp3 = tg_fp3
  fp3 = fp3.view(fp3.size(0), -1)
  if feature_mode:
      return fp3
  else:
      outputs = tg_model.fc(fp3)
      feature = fp3
      return outputs, feature

def itr_merge(*itrs):
    for itr in itrs:
        for v in itr:
            yield v

class ConcatDataset(torch.utils.data.Dataset):
    def __init__(self, *datasets):
        self.datasets = datasets

    def __getitem__(self, i):
        return tuple(d[i] for d in self.datasets)

    def __len__(self):
        return min(len(d) for d in self.datasets)


class mnemonics():
  def __init__(self, randomseed):
    self.model = resnet32(num_classes=100).to('cuda')
    self.feature_extractor = self.model.features
    self.lr = 2
    self.gamma = 0.2
    self.weight_decay = 1e-5 
    self.milestones = [49,63]
    self.batch_size = 128
    self.numepochs = 1
    self.n_classes = 0
    self.n_known = 0
    self.feature_size=64
    self.momentum=0.9
    self.criterion = nn.BCEWithLogitsLoss()

    self.NUM_BATCHES=10
    self.randomseed=randomseed
    self.trainloader=None
    self.testloader=None
    self.CLASSES_PER_BATCH=10

    self.original_training_set = ilCIFAR100(self.CLASSES_PER_BATCH, self.randomseed, train = 'train')

    self.original_exemplar_set = ilCIFAR100(self.CLASSES_PER_BATCH, self.randomseed, train = 'exemplars')

    self.original_test_set = ilCIFAR100(self.CLASSES_PER_BATCH,self.randomseed, train= 'test')

    #self.current_training_set = tempDataset()
    self.exemplar_sets = []
    self.exemplar_labels = []
    self.last_test = None
    self.y_pred = []
    self.y_test = []

    self.cumulative_class_mean = []

    self.classes_seen=0
    self.diz = self.original_training_set.get_dict()

    self.mn_exemplar_means = None
    # lista di liste, ogni lista contiene gli exemplars di una classe
    self.mn_exemplar_sets = [] 
    # lista di liste, ogni lista contiene le labels dell'elemento corrispondente
    self.mn_exemplar_labels = []

  def update_params(self, train_data, trainable_params, epochs):
    pass

  def model_level_optimization(self):
    old_model = copy.deepcopy(self.model)
    old_model.eval()
    old_model.to('cuda')
    n_classes = self.classes_seen+self.CLASSES_PER_BATCH
    print(n_classes)
    optimizer = optim.SGD(self.model.parameters(), lr=self.lr, momentum=self.momentum, weight_decay=self.weight_decay)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=self.milestones, gamma=self.gamma)
    for epoch in tqdm(range(self.numepochs)):
        
      for _, inputs, labels in self.trainloader:
        inputs = inputs.float().cuda()
        labels = torch.tensor([self.diz[c.item()] for c in labels])

        labels=labels.to('cuda')
        optimizer.zero_grad()
        outputs=self.model(inputs)

        labels_encoded = F.one_hot(labels,100).float().cuda() #CAMBIARE ONE_HOT
        
        if self.classes_seen:
          old_target = old_model(inputs).cuda()
          old_target = torch.sigmoid(old_target).cuda()
          
          target = torch.cat((old_target[:,:self.classes_seen], labels_encoded[:, self.classes_seen:]), dim=1)
          loss = self.criterion(outputs, target)
        else:
          loss = self.criterion(outputs,labels_encoded) 

        loss.backward()
        optimizer.step()
      
      scheduler.step()

  def exemplar_level_optimization(self, new_mn_exemplars):
    pass


  def classify_nme(self, input_batch):
    min_distances = float('inf')*torch.ones(len(input_batch)).cuda() # shape: batch_size --> 128
    y_pred = torch.zeros(len(input_batch), dtype=torch.int8).cuda() # shape: batch_size --> 128
    input_features = self.model.features(input_batch) # shape: (batch_size, feature_size) --> (128, 64)

    for i in range(len(self.exemplar_sets)):
      ex_mean = self.exemplar_means[i,:]

      # compute distances between inputs features and exemplar set means
      pdist = nn.PairwiseDistance(p=2)
      distances = pdist(input_features, ex_mean) # shape: batch_size --> 128

      # update min distancies and predicted labels
      mask = distances < min_distances
      min_distances[mask] = distances[mask]
      y_pred[mask] = self.exemplar_labels[i]

    return y_pred
    

  def get_new_exemplars(self, batch, m):
    loader = torch.utils.data.DataLoader(batch, batch_size=self.batch_size,shuffle=False, num_workers=4)
    features = np.zeros((0,self.feature_size))
    indices = np.zeros((0), dtype=int)
    with torch.no_grad():
      for indexes, images, labels in loader:
        images = images.cuda()
        feature = self.feature_extractor(images).data.cpu().numpy()
        feature = normalize(feature, axis=1, norm='l2')
        features = np.concatenate((features,feature), axis=0)
        indices = np.concatenate((indices,indexes), axis=0)

    class_mean = np.mean(features, axis=0)
    class_mean = class_mean / np.linalg.norm(class_mean)  # Normalize

    self.cumulative_class_mean.append(class_mean)

    exemplar_set = []
    exemplar_features = np.zeros((0,self.feature_size))

    for k in range(1, int(m)+1):
        S = np.sum(exemplar_features, axis=0)
        phi = features
        mu = class_mean
        mu_p = 1.0 / k * (phi + S)
        mu_p = normalize(mu_p, axis=1, norm='l2')
        i = np.argmin(np.sqrt(np.sum((mu - mu_p) ** 2, axis=1)))
        exemplar_set.append(int(indices[i]))
        addfeature =  np.expand_dims(features[i], axis=0)
        exemplar_features = np.concatenate((exemplar_features,addfeature), axis=0)

        #remove duplicates
        features = np.delete(features, i, 0)
        indices = np.delete(indices, i, 0)
        
    self.exemplar_sets.append(exemplar_set)
        
  def reduce_old_exemplars(self, m):
    for y, P_y in enumerate(self.exemplar_sets):
            self.exemplar_sets[y] = P_y[:int(m)]


  def __accuracy_fc(self, dl, mapper):
    total = 0.0
    correct = 0.0
    for  _, images, labels in dl:
      labels = torch.tensor([torch.tensor(mapper[c.item()]) for c in labels])
      labels = labels.cuda()
      images = images.cuda()
      outputs = self.model(images)
      _, preds = torch.max(outputs, dim=1)
      total += len(labels)
      correct += torch.sum(preds == labels).item()

    acc = correct / total
    return acc


  def __accuracy_nme(self, dl):
    
    total = 0.0
    correct = 0.0
    
    for  _, images, labels in dl:
      labels = labels.cuda()
      images = images.cuda()
      preds = self.classify_nme(images)
      total += len(labels)
      correct += torch.sum(preds == labels).item()

      if self.last_test:
        self.y_pred += preds.tolist()
        self.y_test += labels.tolist()

    acc = correct / total
    return acc


  def plot_confusion_matrix(self):
 
    cm = confusion_matrix(self.y_test, self.y_pred)
    cm = np.log(cm+1)
    fig, ax = plt.subplots(figsize=(7,7))
    sns.heatmap(cm, square=True, cbar=False, ax=ax, cmap=plt.get_cmap('seismic'))
    ax.set_xticks(np.linspace(19,99,5))
    ax.set_yticks(np.linspace(19,99,5))
    ax.set_xticklabels([20,40,60,80,100], rotation=0)
    ax.set_yticklabels([20,40,60,80,100], rotation=0)
    ax.set_title("iCaRL")
    ax.set_xlabel("Predicted class")
    ax.set_ylabel("True class")
    plt.savefig("iCaRL_"+str(self.randomseed)+"_cm.png")
    plt.show()
    return cm


  
  

  def trainer(self):

    train_indices = self.original_training_set.get_batch_indexes()
    test_indices = self.original_test_set.get_batch_indexes()
    batches=self.original_training_set.getbatches()
    current_test_indexes=[]
    test_acc = []
    self.last_test = False

    for i in range(self.NUM_BATCHES):
      
      if i == self.NUM_BATCHES-1:
        self.last_test = True

      current_exemplar_indices = []
      #for exemplar_set in self.exemplar_sets:
       # train_indices[i]=np.concatenate([train_indices[i], np.array(exemplar_set)])
    
      for exemplar_set in self.exemplar_sets:
        current_exemplar_indices = np.concatenate([current_exemplar_indices, exemplar_set])
        #print(exemplars)

      exemplar_dataset = Subset(self.original_exemplar_set, current_exemplar_indices)
      
      #print('exemplar', exemplar_dataset)
      exemplar_loader = DataLoader(exemplar_dataset, batch_size=self.batch_size, shuffle=False, num_workers=4, drop_last=True)

      train_dataset = Subset(self.original_training_set, train_indices[i])
      current_test_indexes += test_indices[i].tolist()
      test_dataset = Subset(self.original_test_set,current_test_indexes)
      self.train_loader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True, num_workers=4, drop_last=True)
      self.testloader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=False, num_workers=4, drop_last=True)        
     
      #self.model.eval() # Set Network to evaluation mode
      
      if i == 0:
        self.trainloader = self.train_loader
        print('porcoddio')
      else:

        #self.trainloader = itr_merge(self.train_loader, exemplar_loader)
        #self.trainloader = exemplar_loader
        self.trainloader = self.train_loader
        print('len', len(current_exemplar_indices))
        print('indices', current_exemplar_indices)
        print('prova a cazzo', self.original_exemplar_set.__getitem__(int(current_exemplar_indices[0])))
        print('DAL TRAINING SET', self.original_training_set.__getitem__(int(current_exemplar_indices[0])))
        #print('prova a cazzo', self.original_exemplar_set.dataset.data[int(current_exemplar_indices[0])])
        #print('target a cazzo', self.original_exemplar_set.dataset.targets[int(current_exemplar_indices[0])])
        #print('DAL TRAINING SET,', self.original_training_set.dataset.data[int(current_exemplar_indices[0])])
        #print('TARGET DAL TRAINING SET', self.original_training_set.dataset.targets[int(current_exemplar_indices[0])])
      '''
      if i > 0:
        self.trainloader = DataLoader(
              ConcatDataset(train_dataset, exemplar_dataset
              ),
              batch_size=self.batch_size, shuffle=True,
              num_workers=4, pin_memory=True)
      '''

      
      # update exemplars number

      self.model.train()
      self.model_level_optimization()    
      self.classes_seen += 10

      m=int(2000/(int(i*10+10)))

      # reduce the number of each exemplars set
      #self.reduce_old_exemplars(m) 
      
      # randomly choose m indexes for each new class and load the corresponding 
      # data images as mn_exemplars
      current_mn_exemplars = [] # 10 classi, m immagini per clasee, dimesioni delle immagini


      for classlabel in batches[i]:
        indexes_class = self.original_training_set.get_class_indexes(classlabel)
        current_class = Subset(self.original_training_set, indexes_class)
        self.get_new_exemplars(current_class, m)
        
      self.img_size = 32
      self.mnemonics_lrs = 0.01
      num_classes_incremental = 10
      num_classes = 10
      nb_cl = 10
      transform_proto = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5071,  0.4866,  0.4409), (0.2009,  0.1984,  0.2023)),])
      self.mnemonics_label = []
      exemplar_indices = np.array([])
      prototypes = np.zeros((10, m, 3, 32, 32))
      prototypes_label = np.zeros((10,m))
      for xx, exemplar_set in enumerate(self.exemplar_sets[-10:]):
        for j, el in enumerate(exemplar_set):
          prototypes[xx][j] = self.original_training_set.__getitem__(int(el))[1]
          prototypes_label[xx, j] = self.original_training_set.__getitem__(int(el))[2]
      
        #exemplar_indices = np.concatenate([exemplar_indices, np.array(exemplar_set)])
      
  
      self.mnemonics = nn.ParameterList()
      self.mnemonics.append(nn.Parameter(torch.Tensor(prototypes)))
      device = 'cuda'
      self.mnemonics.to(device)
      tg_feature_model = nn.Sequential(*list(self.model.children())[:-1])
      tg_feature_model.eval()
      self.model.eval()

      self.mnemonics_optimizer = optim.SGD(self.mnemonics, lr=self.mnemonics_lrs, momentum=0.9, weight_decay=5e-4)
      self.mnemonics_lr_scheduler = optim.lr_scheduler.StepLR(self.mnemonics_optimizer, step_size=5, gamma=0.2)
      current_means_new = self.cumulative_class_mean
      tg_model = self.model
      start_iteration = 0
      for epoch in range(1):
          
          train_loss = 0
          self.mnemonics_lr_scheduler.step()
          for _, q_inputs, q_targets in self.trainloader:
              q_targets = torch.tensor([self.diz[c.item()] for c in q_targets])
              
              q_inputs, q_targets = q_inputs.to(device), q_targets.to(device)
              if i == start_iteration:
                  q_feature = tg_feature_model(q_inputs)
              else:
                  q_feature = process_inputs_fp(tg_model, q_inputs, feature_mode=True)
              self.mnemonics_optimizer.zero_grad()
              total_tr_loss = 0 
              if i == start_iteration:
                  mnemonics_outputs = tg_feature_model(self.mnemonics[0][0])
              else:
                  mnemonics_outputs = process_inputs_fp(tg_model, self.mnemonics[0][0], feature_mode=True)
              this_class_mean_mnemonics = torch.mean(mnemonics_outputs, dim=0)
              this_class_mean_mnemonics = torch.squeeze(this_class_mean_mnemonics)
              total_class_mean_mnemonics = this_class_mean_mnemonics.unsqueeze(dim=0)
              for mnemonics_idx in range(len(self.mnemonics[0])-1):
                  if i == start_iteration:
                      mnemonics_outputs = tg_feature_model(self.mnemonics[0][mnemonics_idx+1])
                  else:
                      mnemonics_outputs = process_inputs_fp(tg_model, self.mnemonics[0][mnemonics_idx+1], feature_mode=True)
                  this_class_mean_mnemonics = torch.mean(mnemonics_outputs, dim=0)
                  this_class_mean_mnemonics = torch.squeeze(this_class_mean_mnemonics)
                  total_class_mean_mnemonics =  torch.cat((total_class_mean_mnemonics, this_class_mean_mnemonics.unsqueeze(dim=0)), dim=0)
              if i == start_iteration:
                  all_cls_means = total_class_mean_mnemonics
              else:
                  all_cls_means = torch.tensor(current_means_new).float().to(device)
                  all_cls_means[-nb_cl:] = total_class_mean_mnemonics
              the_logits = F.linear(F.normalize(torch.squeeze(q_feature), p=2,dim=1), F.normalize(all_cls_means, p=2, dim=1))
              loss = F.cross_entropy(the_logits, q_targets)
              #loss = nn.CrossEntropyLoss(the_logits, q_targets)
              loss.backward()
              self.mnemonics_optimizer.step()
              train_loss += loss.item()

      #PROVA A VEDERE SE MODIFICANDO IL __GETIMEM__ cambia qualcosa
      for xx, exemplar_set in enumerate(self.exemplar_sets[-10:]):
        for j, el in enumerate(exemplar_set):
          self.original_exemplar_set.dataset.data[int(el)] = tensor2im(self.mnemonics[0][xx][j]) #DOVE SELF:MNEMONICS E MODIFICATO
'''
      #PER NME CLASSIFIER
      # compute means of exemplar set
      # cycle for each exemplar set
      self.exemplar_means = torch.zeros((0, self.feature_size), dtype=torch.float).cuda()
      self.exemplar_labels = []
      for i in range(len(self.exemplar_sets)):
        exemplars_dataset = Subset(self.original_training_set, self.exemplar_sets[i])
        exemplars_loader = torch.utils.data.DataLoader(exemplars_dataset, batch_size=self.batch_size, shuffle=False, num_workers=4)
        ex_features = torch.zeros((0, self.feature_size), dtype=torch.float).cuda() # alla fine shape: (len(exemplar_set), feature_size) --> (m, 64)
      
        with torch.no_grad():
          _, _, exemplar_label = self.original_training_set.__getitem__(self.exemplar_sets[i][0]) 
          self.exemplar_labels.append(exemplar_label)
          # cycle for each batch in the current exemplar set
          for _,  exemplars, _ in exemplars_loader:
          
            # get exemplars features
            exemplars = exemplars.cuda()
            features = self.model.features(exemplars) # shape: (len(exemplars), feature_size)
          
            # normalize 
            feature_norms = torch.norm(features, p=2, dim=1) # shape: len(exemplars)
            feature_norms.unsqueeze_(1) # shape: (len(exemplars), 1)
            features = features/feature_norms
          
            # concatenate over columns
            ex_features = torch.cat((ex_features, features), dim=0)
          
        # compute current exemplar set mean and normalize it
        ex_mean = torch.mean(ex_features, dim=0) # shape: feature_size --> 64
        ex_mean = ex_mean/torch.norm(ex_mean)
        ex_mean.unsqueeze_(0) # shape: (1, feature_size) --> (1, 64)
        self.exemplar_means = torch.cat((self.exemplar_means, ex_mean), dim=0) # shape: (n_examplar set, feature size)
      

      print('accuracy on training set:', 100*self.__accuracy_fc(self.trainloader,self.diz))
      # print('accuracy on test set:', self.__accuracy_on(self.testloader,self,self.diz))
      current_test_acc = self.__accuracy_nme(self.testloader, last_test)
      print('accuracy on test set:', 100*current_test_acc)
      print('-' * 80)
      test_acc.append(current_test_acc)

    # compute comfusion matrix and save results
    cm = self.plot_confusion_matrix()
    with open('iCaRL_'+str(self.randomseed)+"_cm", 'wb') as file:
      pickle.dump(cm, file, protocol=pickle.HIGHEST_PROTOCOL)
    with open('iCaRL_'+str(self.randomseed)+"_testacc", 'wb') as file:
      pickle.dump(test_acc, file, protocol=pickle.HIGHEST_PROTOCOL)
'''

'\n      #PER NME CLASSIFIER\n      # compute means of exemplar set\n      # cycle for each exemplar set\n      self.exemplar_means = torch.zeros((0, self.feature_size), dtype=torch.float).cuda()\n      self.exemplar_labels = []\n      for i in range(len(self.exemplar_sets)):\n        exemplars_dataset = Subset(self.original_training_set, self.exemplar_sets[i])\n        exemplars_loader = torch.utils.data.DataLoader(exemplars_dataset, batch_size=self.batch_size, shuffle=False, num_workers=4)\n        ex_features = torch.zeros((0, self.feature_size), dtype=torch.float).cuda() # alla fine shape: (len(exemplar_set), feature_size) --> (m, 64)\n      \n        with torch.no_grad():\n          _, _, exemplar_label = self.original_training_set.__getitem__(self.exemplar_sets[i][0]) \n          self.exemplar_labels.append(exemplar_label)\n          # cycle for each batch in the current exemplar set\n          for _,  exemplars, _ in exemplars_loader:\n          \n            # get exemplars fea

In [63]:
model = mnemonics(randomseed=203)
model.trainer()

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
porcoddio
10


/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


len 2000
indices [19396. 47252. 21137. ... 15528. 47703. 24546.]
prova a cazzo (19396, tensor([[[-0.3857, -0.1658,  0.3473,  ..., -1.8957, -1.8957, -1.8957],
         [-0.3711, -0.4150, -0.4590,  ..., -1.8957, -1.8957, -1.8957],
         [-0.2391, -0.3417, -0.4297,  ..., -1.8957, -1.8957, -1.8957],
         ...,
         [-1.8957, -1.8957, -1.8957,  ..., -1.8957, -1.8957, -1.8957],
         [-1.8957, -1.8957, -1.8957,  ..., -1.8957, -1.8957, -1.8957],
         [-1.8957, -1.8957, -1.8957,  ..., -1.8957, -1.8957, -1.8957]],

        [[-0.4450, -0.1851,  0.3958,  ..., -1.8975, -1.8975, -1.8975],
         [-0.4145, -0.4450, -0.4450,  ..., -1.8975, -1.8975, -1.8975],
         [-0.2616, -0.3380, -0.3992,  ..., -1.8975, -1.8975, -1.8975],
         ...,
         [-1.8975, -1.8975, -1.8975,  ..., -1.8975, -1.8975, -1.8975],
         [-1.8975, -1.8975, -1.8975,  ..., -1.8975, -1.8975, -1.8975],
         [-1.8975, -1.8975, -1.8975,  ..., -1.8975, -1.8975, -1.8975]],

        [[-0.5029, -0.3040,  

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


len 3000
indices [19396. 47252. 21137. ...  8145.  3238. 24774.]
prova a cazzo (19396, tensor([[[-0.3857, -0.1658,  0.3473,  ..., -1.8957, -1.8957, -1.8957],
         [-0.3711, -0.4150, -0.4590,  ..., -1.8957, -1.8957, -1.8957],
         [-0.2391, -0.3417, -0.4297,  ..., -1.8957, -1.8957, -1.8957],
         ...,
         [-1.8957, -1.8957, -1.8957,  ..., -1.8957, -1.8957, -1.8957],
         [-1.8957, -1.8957, -1.8957,  ..., -1.8957, -1.8957, -1.8957],
         [-1.8957, -1.8957, -1.8957,  ..., -1.8957, -1.8957, -1.8957]],

        [[-0.4450, -0.1851,  0.3958,  ..., -1.8975, -1.8975, -1.8975],
         [-0.4145, -0.4450, -0.4450,  ..., -1.8975, -1.8975, -1.8975],
         [-0.2616, -0.3380, -0.3992,  ..., -1.8975, -1.8975, -1.8975],
         ...,
         [-1.8975, -1.8975, -1.8975,  ..., -1.8975, -1.8975, -1.8975],
         [-1.8975, -1.8975, -1.8975,  ..., -1.8975, -1.8975, -1.8975],
         [-1.8975, -1.8975, -1.8975,  ..., -1.8975, -1.8975, -1.8975]],

        [[-0.5029, -0.3040,  

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


len 3660
indices [19396. 47252. 21137. ... 27230. 32890. 15993.]
prova a cazzo (19396, tensor([[[-0.3857, -0.1658,  0.3473,  ..., -1.8957, -1.8957, -1.8957],
         [-0.3711, -0.4150, -0.4590,  ..., -1.8957, -1.8957, -1.8957],
         [-0.2391, -0.3417, -0.4297,  ..., -1.8957, -1.8957, -1.8957],
         ...,
         [-1.8957, -1.8957, -1.8957,  ..., -1.8957, -1.8957, -1.8957],
         [-1.8957, -1.8957, -1.8957,  ..., -1.8957, -1.8957, -1.8957],
         [-1.8957, -1.8957, -1.8957,  ..., -1.8957, -1.8957, -1.8957]],

        [[-0.4450, -0.1851,  0.3958,  ..., -1.8975, -1.8975, -1.8975],
         [-0.4145, -0.4450, -0.4450,  ..., -1.8975, -1.8975, -1.8975],
         [-0.2616, -0.3380, -0.3992,  ..., -1.8975, -1.8975, -1.8975],
         ...,
         [-1.8975, -1.8975, -1.8975,  ..., -1.8975, -1.8975, -1.8975],
         [-1.8975, -1.8975, -1.8975,  ..., -1.8975, -1.8975, -1.8975],
         [-1.8975, -1.8975, -1.8975,  ..., -1.8975, -1.8975, -1.8975]],

        [[-0.5029, -0.3040,  

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/container.py:434: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


KeyboardInterrupt: ignored

In [10]:
x = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
for i, el in enumerate(x):
  if i < 10:
    print(el)

1
2
3
4
5
6
7
8
9
10


In [12]:
x[-20:]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]